In [ ]:
import cv2
import numpy as np
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Load the input image
input_image_path = '/content/drive/My Drive/IMG_5450.jpg'  # Update with the actual path to your image
try:
    input_image = cv2.imread(input_image_path)
    if input_image is None:
        raise FileNotFoundError("Input image not found or unable to load.")
    else:
        print("Input image loaded successfully.")
        print("Image type:", type(input_image))
        print("Image shape:", input_image.shape)
except Exception as e:
    print("Error loading input image:", e)
    exit()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Input image loaded successfully.
Image type: <class 'numpy.ndarray'>
Image shape: (3024, 4032, 3)


In [ ]:
# Threshold-based segmentation algorithm
def threshold_segmentation(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 120, 255, cv2.THRESH_BINARY)
    return binary_image

In [ ]:
# Watershed segmentation algorithm
def watershed_segmentation(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray_image, (5, 5), 0)
    # Thresholding
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # Morphological opening
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    # Background and foreground masks
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    # Marker labelling
    _, markers = cv2.connectedComponents(sure_fg)
    markers += 1
    markers[unknown == 255] = 0
    # Watershed transformation
    markers = cv2.watershed(image, markers)
    image[markers == -1] = [0, 0, 255]
    return image


In [ ]:
# Apply segmentation algorithms
threshold_result = threshold_segmentation(input_image)
watershed_result = watershed_segmentation(input_image)

In [ ]:
from google.colab.patches import cv2_imshow

# Get the size of the threshold result image
threshold_size = threshold_result.shape[:2]  # Height and width only

# Get the size of the watershed result image
watershed_size = watershed_result.shape[:2]  # Height and width only

# Display the sizes
print("Threshold Segmentation Result Size:", threshold_size)
print("Watershed Segmentation Result Size:", watershed_size)

# Display results using cv2_imshow
cv2_imshow(threshold_result)
cv2_imshow(watershed_result)

